In [6]:
from zai import ZhipuAiClient
# 初始化客户端
client = ZhipuAiClient(api_key="70592327bc39433a8ae11a37b0d6a814.9JBJm2z4KXYnAdqu")

In [13]:
query = """
你将收到一段投资者在公司业绩说明会中提出的问题。

**任务**
请判断该问题表达的情感倾向属于以下哪一类：
- 积极
- 消极
- 中性

**输出**
仅返回一个词语：“积极”、“消极”或“中性”。

**问题：**
"""

In [14]:
def get_emotion(text):
    # 创建聊天完成请求
    response = client.chat.completions.create(
        model="glm-4.5-flash",
        messages=[
            {
                "role": "system",
                "content": "你是一名情感分析助手。"
            },
            {
                "role": "user",
                "content": query + text
            }
        ],
        temperature=0.6
    )
    # 获取回复
    return response.choices[0].message.content

In [1]:
import pandas as pd

In [5]:
#导入excel
xls_file = pd.ExcelFile('./data/PES_Interaction.xlsx')
df = xls_file.parse('Sheet1', index_col=None, na_values=['NA'], converters={'Symbol': str})
df = df.loc[:, ['ReportDate', 'Symbol', 'QuestionContent']].copy()
tb = df.sample(10).copy()
tb

,ReportDate,Symbol,QuestionContent
27939,2025-09-18,688138,请问公司的掩膜版目前主要生产的规格是多少nm？是否可以量产更小规格的。
31529,2025-05-23,688611,提问：高管您好，请问贵公司未来盈利增长的主要驱动因素有哪些？谢谢。
25631,2025-05-30,605020,贵公司在六氟环氧丙烷主要用于什么领域，目前产能多少？
13368,2025-09-25,600568,请问公司子公司深圳市一体医疗科技有限公司目前经营是否正常？
4817,2025-09-12,002838,请问中试线有没有什么问题，能够足量产出5000顿吗？扩展到2万吨有没有什么障碍？5000顿后...
5771,2025-09-19,300180,中报看超纤去掉尼龙6营收的情况下，超纤业务大福萎缩，是什么原因造成贵公司选择了低毛利的尼龙增...
9492,2025-09-10,600100,据说同方股份作为中核集团的数字信息产业发展平台和信息技术支撑总体单位，牵头承接了中核集团数字...
16509,2025-09-19,600895,请问：未来会不会开展在半导体和装备等制造业？
24347,2025-09-26,603773,大基金三期已经开始投资，据传大基金已经考察了沃格光电（通格微）请问是否真实。
30104,2025-06-11,688391,杨董您好，贵公司的MCU条线毛利率持续下降，今年是否有止跌趋势？是否有意进行车规级MCU产品...


In [16]:
from tqdm import tqdm

In [17]:
for index, row in tqdm(tb.iterrows(), total=tb.shape[0]):
    QuestionContent = row.QuestionContent
    tb.loc[index, 'emotion'] = get_emotion(QuestionContent)
#     print(QuestionContent)
#     print(get_emotion(QuestionContent))
#     print('\n==================================\n')

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:07<00:00,  6.74s/it]


In [18]:
tb

,ReportDate,Symbol,QuestionContent,emotion
27939,2025-09-18,688138,请问公司的掩膜版目前主要生产的规格是多少nm？是否可以量产更小规格的。,中性
31529,2025-05-23,688611,提问：高管您好，请问贵公司未来盈利增长的主要驱动因素有哪些？谢谢。,中性
25631,2025-05-30,605020,贵公司在六氟环氧丙烷主要用于什么领域，目前产能多少？,中性
13368,2025-09-25,600568,请问公司子公司深圳市一体医疗科技有限公司目前经营是否正常？,中性
4817,2025-09-12,002838,请问中试线有没有什么问题，能够足量产出5000顿吗？扩展到2万吨有没有什么障碍？5000顿后...,中性
5771,2025-09-19,300180,中报看超纤去掉尼龙6营收的情况下，超纤业务大福萎缩，是什么原因造成贵公司选择了低毛利的尼龙增...,消极
9492,2025-09-10,600100,据说同方股份作为中核集团的数字信息产业发展平台和信息技术支撑总体单位，牵头承接了中核集团数字...,\n中性
16509,2025-09-19,600895,请问：未来会不会开展在半导体和装备等制造业？,中性
24347,2025-09-26,603773,大基金三期已经开始投资，据传大基金已经考察了沃格光电（通格微）请问是否真实。,中性
30104,2025-06-11,688391,杨董您好，贵公司的MCU条线毛利率持续下降，今年是否有止跌趋势？是否有意进行车规级MCU产品...,\n中性
